In [ ]:
import src
from pathlib import Path
from itertools import repeat

import torch
from torch.nn import functional as F
from tqdm.notebook import trange, tqdm

from src import Petri_Cheb, Petri_GCN, Petri_SAGE

batch_size = 64
EPOCHS = 100
train_dataset = src.get_reachability_dataset(Path('Data/RandData_DS1_train_data.processed'), batch_size=batch_size)
test_dataset = src.get_reachability_dataset(Path('Data/RandData_DS1_test_data.processed'), batch_size=batch_size)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train(model, optimizer):
    model.train()
    for epoch in trange(EPOCHS, leave=False):
        for graph in train_dataset:
            optimizer.zero_grad()
            out = torch.flatten(model(graph))
            loss = F.l1_loss(out, graph.y)
            loss.backward()
            optimizer.step()

In [ ]:
def test(model):
    model.eval()
    pred = []
    actual = torch.tensor([graph.y for graph in test_dataset.dataset])
    for graph in test_dataset:
        pred.extend(model(graph).tolist())
    
    pred = torch.tensor(pred)
    pred = torch.flatten(pred)

    mre = torch.mean(F.l1_loss(pred, actual, reduction='none') / actual) * 100
    return F.l1_loss(pred, actual), torch.nn.MSELoss()(pred, actual), mre

In [ ]:
def run_scenario(create_model, scenario, compile=False):
    values = []
    for i in trange(30, leave=False, desc=scenario):
        model = create_model()
        if compile:
            model = torch.compile(model, dynamic=True)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
        train(model, optimizer)
        values.append(test(model))
    values = torch.tensor(values)
    return values

In [ ]:
GNN_layers = 5
MLP_layers = 2
pairs = repeat((GNN_layers, MLP_layers))
GNN_Operators = [("GCN", Petri_GCN), ("Chebyshev", Petri_Cheb), ("SAGE", Petri_SAGE)]
results = []
for (name, operator), layers in tqdm(list(zip(GNN_Operators, pairs))):
    scenario = f'{name}'
    f = lambda: operator(train_dataset.num_features, 16, num_layers=layers[0], readout_layers=layers[1])
    scenario_result = run_scenario(f, scenario)
    results.append((scenario, scenario_result))

In [ ]:
dir = Path('./results')
if not dir.exists():
    dir.mkdir()
for scenario, result in results:
    final_path = dir / f"{scenario}.pt"
    torch.save(result, final_path)

In [ ]:
def variance(sample):
    return torch.mean((sample - torch.mean(sample))**2)

def std_deviation(sample):
    return variance(sample)**2

def statistical_values(sample):
    return torch.mean(sample), std_deviation(sample), variance(sample)

In [ ]:
statistics = []
for scenario, results in read_tensor_data():
    mae = results[:, 0]
    mse = results[:, 1]
    mre = results[:, 2]
    print(f"Values for scenario: {scenario}\n")
    print("MAE\tMean: %.5f\ts²: %.5f\ts: %.5f\n".format(*statistical_values(mae)))
    print("MSE\tMean: %.5f\ts²: %.5f\ts: %.5f\n".format(*statistical_values(mae)))
    print("MRE\tMean: %.5f\ts²: %.5f\ts: %.5f\n".format(*statistical_values(mae)))
    print("-"*80)
    statistics.append((scenario, list(map(statistical_values, [mae, mse, mre])))